## 定期預金の契約についてのデータセット

データの取得
- https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip<br>
bank.csv を使用

データの説明
- https://archive.ics.uci.edu/ml/datasets/bank+marketing

内容を確認
- "age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"<br>
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
- "y" 定期預金を預けてくれたかどうか

In [1]:
import pandas as pd
import numpy as np

bank = pd.read_csv('data/bank.csv', sep=';')

# 使用する列のみからなる DataFrame を作成
bank = bank[['y', 'job', 'campaign']]

# 'y'列のダミー変数化: yes->1, no->0
bank['y'] = bank['y'].map(lambda x: 1 if x == 'yes' else 0)

# 列ごとの欠損値の数
bank.isna().sum()

y           0
job         0
campaign    0
dtype: int64

In [2]:
bank.head(2)

,y,job,campaign
0,0,unemployed,1
1,0,services,1


- y: 定期預金を契約したか（目的変数）
- job: 職種
- campaign: 勧誘のコンタクト回数

In [3]:
# job列の retired, student, unemployed, unknown 
# は無職、それ以外を有職とした時、有職者の数を答えない。（整数のため小数点以下は不要）

bank.query("job != ['retired', 'student', 'unemployed', 'unknown']").count()

y           4041
job         4041
campaign    4041
dtype: int64

In [4]:
# job列は上記の場合分けで有職 (employed=1), 無職 (employed=0)とするダミー変数化を行った employed 列を追加し、job列は削除する。
unemployed_arr = ['retired', 'student', 'unemployed', 'unknown']
bank['employed'] = bank['job'].map(lambda x: 0 if x in unemployed_arr else 1)
bank

bank.drop('job', axis=1)

# そのうえで、y を目的変数、campaign, employed を説明変数として、ロジスティック回帰モデルを作成し、AUCを算出しなさい。
# 小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること


,y,campaign,employed
0,0,1,0
1,0,1,1
2,0,1,1
3,0,4,1
4,0,1,1
...,...,...,...
4516,0,5,1
4517,0,1,1
4518,0,11,1
4519,0,4,1


In [5]:
# そのうえで、y を目的変数、campaign, employed を説明変数として、ロジスティック回帰モデルを作成し、AUCを算出しなさい。
# 小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

X = bank[['campaign', 'employed']]
Y = bank.y

model = LogisticRegression(C=10000.0)
model.fit(X, Y)
s = model.score(X, Y)

Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
print(auc(fpr, tpr))
print(s)

0.5785962092130518
0.8847600088476001


In [6]:
# y を目的変数、campaign のみを説明変数として作成したロジスティック回帰モデルを分析し、
# 勧誘のコンタクト回数が増えると契約がとれる確率は上がるかどうか答えなさい。
X = bank[['campaign']]
Y = bank.y

model = LogisticRegression(C=10000.0)
model.fit(X, Y)
s = model.score(X, Y)

Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
print(auc(fpr, tpr))
print(s)

0.5560083973128599
0.8847600088476001
